# Official IMDB Data Analysis

- Overview/Data Dictionary: https://www.imdb.com/interfaces/

## Objective

**Specifications**

Your stakeholder only wants you to include information for movies based on the following specifications:

- Exclude any movie with missing values for genre or runtime
- Include only full-length movies (titleType = "movie").
- Include only fictional movies (not from documentary genre)
- Include only movies that were released 2000 - 2021 (include 2000 and 2021)
- Include only movies that were released in the United States


**Deliverable**

After filtering out movies that do not meet the stakeholder's specifications:

- Before saving, run a final .info() for each of the dataframes to show a summary of how many movies remain and the datatypes of each feature
- Save each file to a compressed csv file "Data/" folder inside your repository.
- Commit your changes to your repository in GitHub desktop and Publish repository / Push Changes.
- Submit the link to your repository

## Imports

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import os

## Data

In [2]:
# Basics
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

# Akas
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

# Ratings
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

### Basics Data

In [3]:
# Display the first 5 rows of the basics dataframe
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
# Display the column names, count of non-null values, and their datatypes
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9963680 entries, 0 to 9963679
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 684.2+ MB


In [5]:
# Display the descriptive statistics for all columns
basics.describe()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
count,9963680,9963680,9963669,9963669,9963680,9963680,9963680,9963680,9963665
unique,9963680,11,4499453,4521584,11,154,96,892,2342
top,tt0000001,tvEpisode,Episode #1.1,Episode #1.1,0,\N,\N,\N,Drama
freq,1,7576582,48176,48176,9650424,1342867,9854523,7011782,1135182


In [6]:
# Display the number of duplicate rows in the dataset
print(f'There are {basics.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


In [7]:
# Display the total number of missing values
print(f'There are {basics.isna().sum().sum()} missing values.')

There are 37 missing values.


In [8]:
# Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace = True)

In [9]:
# Eliminate movies that are null for runtimeMinutes & genres
basics.dropna(subset=['runtimeMinutes','genres'], inplace = True)

In [10]:
# Keep only titleType==Movie
basics = basics.loc[basics['titleType']=='movie']

In [11]:
# Check categories left in column titleType
basics['titleType'].value_counts()

movie    384408
Name: titleType, dtype: int64

In [12]:
# Keep startYear 2000-2022
# from .info, its observed its an object, so ill change dtype to int 
basics.dropna (subset = ["startYear"], inplace = True)
basics['startYear'] = basics['startYear'].astype(int)

# Apply two filters to make startYear between 2000 to 2022
basics = basics[(basics['startYear'] >= 2000) & (basics['startYear'] <= 2022)]
basics['startYear'].value_counts()

2017    14388
2018    14357
2019    14116
2016    13974
2015    13484
2014    13128
2022    12970
2021    12434
2013    12397
2012    11654
2020    11602
2011    10783
2010    10216
2009     9372
2008     8167
2007     6972
2006     6528
2005     5852
2004     5219
2003     4601
2002     4140
2001     3879
2000     3648
Name: startYear, dtype: int64

In [13]:
# Eliminate movies that include "Documentary" in genre 
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [14]:
print (basics)

            tconst titleType  \
34803    tt0035423     movie   
42384    tt0043139     movie   
61115    tt0062336     movie   
67667    tt0069049     movie   
86799    tt0088751     movie   
...            ...       ...   
9963353  tt9916170     movie   
9963362  tt9916190     movie   
9963401  tt9916270     movie   
9963446  tt9916362     movie   
9963530  tt9916538     movie   

                                              primaryTitle  \
34803                                       Kate & Leopold   
42384                          Life of a Beijing Policeman   
61115    The Tango of the Widower and Its Distorting Mi...   
67667                           The Other Side of the Wind   
86799                                    The Naked Monster   
...                                                    ...   
9963353                                      The Rehearsal   
9963362                                          Safeguard   
9963401                           Il talento del calabron

### AKAs Data

In [15]:
# Display the first 5 rows of the akas dataframe
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [16]:
# Display the column names, count of non-null values, and their datatypes
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36370992 entries, 0 to 36370991
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [17]:
# Display the descriptive statistics for all columns
akas.describe()

,ordering
count,3.637099e+07
mean,4.148210e+00
std,3.932816e+00
min,1.000000e+00
25%,2.000000e+00
50%,4.000000e+00
75%,6.000000e+00
max,2.490000e+02


In [18]:
# Display the number of duplicate rows in the dataset
print(f'There are {akas.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


In [19]:
# Display the total number of missing values
print(f'There are {akas.isna().sum().sum()} missing values.')

There are 119 missing values.


In [20]:
# Replace "\N" with np.nan
akas.replace({'\\N': np.nan}, inplace=True)

In [21]:
# Keep only US movies
# drop nulls
akas.dropna (subset = ['region'], inplace = True)
# Apply filter
akas = akas[akas['region'] == 'US']
akas['region'].value_counts()

US    1447561
Name: region, dtype: int64

In [22]:
print (basics)

            tconst titleType  \
34803    tt0035423     movie   
42384    tt0043139     movie   
61115    tt0062336     movie   
67667    tt0069049     movie   
86799    tt0088751     movie   
...            ...       ...   
9963353  tt9916170     movie   
9963362  tt9916190     movie   
9963401  tt9916270     movie   
9963446  tt9916362     movie   
9963530  tt9916538     movie   

                                              primaryTitle  \
34803                                       Kate & Leopold   
42384                          Life of a Beijing Policeman   
61115    The Tango of the Widower and Its Distorting Mi...   
67667                           The Other Side of the Wind   
86799                                    The Naked Monster   
...                                                    ...   
9963353                                      The Rehearsal   
9963362                                          Safeguard   
9963401                           Il talento del calabron

### Ratings Data

In [23]:
# Display the first 5 rows of the ratings dataframe
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1839
3,tt0000004,5.5,178
4,tt0000005,6.2,2625


In [24]:
# Display the column names, count of non-null values, and their datatypes
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324473 entries, 0 to 1324472
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1324473 non-null  object 
 1   averageRating  1324473 non-null  float64
 2   numVotes       1324473 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.3+ MB


In [25]:
# Display the descriptive statistics for all columns
ratings.describe()

,averageRating,numVotes
count,1.324473e+06,1.324473e+06
mean,6.954582e+00,1.038616e+03
std,1.382727e+00,1.746931e+04
min,1.000000e+00,5.000000e+00
25%,6.200000e+00,1.100000e+01
50%,7.100000e+00,2.600000e+01
75%,7.900000e+00,1.010000e+02
max,1.000000e+01,2.757458e+06


In [26]:
# Display the number of duplicate rows in the dataset
print(f'There are {ratings.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


In [27]:
# Display the total number of missing values
print(f'There are {ratings.isna().sum().sum()} missing values.')

There are 0 missing values.


In [28]:
# Replace "\N" with np.nan
ratings.replace({'\\N': np.nan}, inplace=True)

In [29]:
# Keep only US movies
# Filter the ratings table down to only include the US by 
# using the filter akas dataframe
ratings_keepers =ratings['tconst'].isin(akas['titleId'])

# Filter ratings
ratings = ratings[ratings_keepers]

In [30]:
print (ratings)

            tconst  averageRating  numVotes
0        tt0000001            5.7      1982
1        tt0000002            5.8       265
4        tt0000005            6.2      2625
5        tt0000006            5.1       182
6        tt0000007            5.4       820
...            ...            ...       ...
1324434  tt9916200            8.1       230
1324435  tt9916204            8.2       264
1324442  tt9916348            8.3        18
1324443  tt9916362            6.4      5390
1324448  tt9916428            3.8        14

[502223 rows x 3 columns]


# Save Files into Repository

In [31]:
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

[]

In [32]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [33]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [34]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000005,6.2,2625
3,tt0000006,5.1,182
4,tt0000007,5.4,820
